In [1]:
library(stringr)
library(data.table) #rbindlist
library(purrr) #map
library(dplyr)
library(hash) #dico


Attaching package: 'purrr'

The following object is masked from 'package:data.table':

    transpose


Attaching package: 'dplyr'

The following objects are masked from 'package:data.table':

    between, first, last

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

hash-2.2.6 provided by Decision Patterns


Attaching package: 'hash'

The following object is masked from 'package:data.table':

    copy



In [2]:
path_extracted_data = 'data/extracted_data_R'

In [3]:
START_ALL = Sys.time()

# Define parameters

In [4]:
#number of seconds that a hens needs to stay at least in a zone to count as a true transition
#note that if nbr_sec =3, then from 2019-07-07 22:09:32 to 2019-07-07 22:09:35 , there is at least 2 seconds so its consider
#as a transition, but not 32 - 34 
nbr_sec = 3

#choose folder names
#path to initial data
path_initial_data = "data/initial_data"
#define & create path to extracted data
path_extracted_data = "data/extracted_data"

# Download initial data

In [5]:
start = Sys.time()
temp = list.files(path="data/initial_data",pattern="log_*", full.names=TRUE) 
df = lapply(temp, function(x){read.csv(file=x, header=FALSE, sep=';')}) 
#add the order of each csv
#df_ = lapply(df_, function(x) {cbind(x, ts_order = seq.int(nrow(x)))})
end = Sys.time()
print(paste0("Total running time: ", (end-start)/60," mn"))

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
"line 1 appears to contain embedded nulls"Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
"incomplete final line found by readTableHeader on 'data/initial_data/Video_logfiles_Comparison.xlsx'"

[1] "Total running time: 0.533348651727041 mn"


In [6]:
#merge each csv
start = Sys.time()
df = rbindlist(df, fill=TRUE)
end = Sys.time()
print(paste0("Total running time: ", (end-start)/60," mn"))

[1] "Total running time: 0.0802405476570129 mn"


In [7]:
#add column names
names(df) = c('Timestamp','Serialnumber_Tag','TagID_Pen_HenID','Zone','Signal','U1','U2')

In [8]:
print(dim(df))
head(df)

[1] 6971572       7


Timestamp,Serialnumber_Tag,TagID_Pen_HenID,Zone,Signal,U1,U2
07.07.2019 22:09:28,02001560,060 4B,Rampe 1-5,12,2,2
07.07.2019 22:09:28,020015C6,067 4L,Rampe 1-5,14,2,2
07.07.2019 22:09:29,02001560,060 4B,Einstreu 1-5,12,2,1
07.07.2019 22:09:29,020015C6,067 4L,Einstreu 1-5,14,2,1
07.07.2019 22:09:29,02001560,060 4B,Rampe 1-5,12,2,2
07.07.2019 22:09:29,020015C6,067 4L,Rampe 1-5,14,2,2


# Data preprocessing

#### Remove dates with health care

In [9]:
#add day variable and put date into as.POSIXct format
start = Sys.time()
df$day = as.Date(df$Timestamp, format="%d.%m.%Y %H:%M:%S")
df$Timestamp = as.POSIXct(df$Timestamp, format="%d.%m.%Y %H:%M:%S")
end = Sys.time()
print(paste0("Total running time: ", (end-start)/60," mn"))

[1] "Total running time: 0.028956330815951 mn"


In [10]:
print(dim(df))
head(df)

[1] 6971572       8


Timestamp,Serialnumber_Tag,TagID_Pen_HenID,Zone,Signal,U1,U2,day
2019-07-07 22:09:28,02001560,060 4B,Rampe 1-5,12,2,2,2019-07-07
2019-07-07 22:09:28,020015C6,067 4L,Rampe 1-5,14,2,2,2019-07-07
2019-07-07 22:09:29,02001560,060 4B,Einstreu 1-5,12,2,1,2019-07-07
2019-07-07 22:09:29,020015C6,067 4L,Einstreu 1-5,14,2,1,2019-07-07
2019-07-07 22:09:29,02001560,060 4B,Rampe 1-5,12,2,2,2019-07-07
2019-07-07 22:09:29,020015C6,067 4L,Rampe 1-5,14,2,2,2019-07-07


In [11]:
#create list of date to be removed given by Yamenah
li_date2remove = c(as.Date('2019-07-09'), as.Date('2019-07-10'), as.Date('2019-07-16'), as.Date('2019-07-17'), 
                   as.Date('2019-07-23'), as.Date('2019-07-24'), as.Date('2019-07-30'), as.Date('2019-07-31'), 
                   as.Date('2019-08-06'), as.Date('2019-08-07'), as.Date('2019-08-13'), as.Date('2019-08-14'), 
                   as.Date('2019-08-19'), as.Date('2019-08-20'), as.Date('2019-08-21'))

In [12]:
#filter the dates
df = filter(df, !(day %in% li_date2remove))

In [13]:
print(dim(df))
head(df)

[1] 4650744       8


Timestamp,Serialnumber_Tag,TagID_Pen_HenID,Zone,Signal,U1,U2,day
2019-07-07 22:09:28,02001560,060 4B,Rampe 1-5,12,2,2,2019-07-07
2019-07-07 22:09:28,020015C6,067 4L,Rampe 1-5,14,2,2,2019-07-07
2019-07-07 22:09:29,02001560,060 4B,Einstreu 1-5,12,2,1,2019-07-07
2019-07-07 22:09:29,020015C6,067 4L,Einstreu 1-5,14,2,1,2019-07-07
2019-07-07 22:09:29,02001560,060 4B,Rampe 1-5,12,2,2,2019-07-07
2019-07-07 22:09:29,020015C6,067 4L,Rampe 1-5,14,2,2,2019-07-07


#### TagID_Pen_HenID variable

In [14]:
#remove records associated to '15C3' HenID
start = Sys.time()
#replace'_' by ''
df$TagID_Pen_HenID = lapply(df$TagID_Pen_HenID, function(x){gsub("_", " ", x)})
x0 = dim(df)
df = df[df$TagID_Pen_HenID!='15C3', ]
#remove record associated to wrong HenID
end = Sys.time()
print(paste0("Total running time: ", (end-start)/60," mn"))
paste0("we lost ", x0[[1]]-dim(df)[[1]], " records due to TagID_Pen_HenID correction, we are left with ", 
       dim(df)[[1]], ' records')

[1] "Total running time: 0.025981818901168 mn"


[1] "we lost 5766 records due to TagID_Pen_HenID correction, we are left with 4644978 records"

In [15]:
print(dim(df))
head(df)

[1] 4644978       8


Timestamp,Serialnumber_Tag,TagID_Pen_HenID,Zone,Signal,U1,U2,day
2019-07-07 22:09:28,02001560,060 4B,Rampe 1-5,12,2,2,2019-07-07
2019-07-07 22:09:28,020015C6,067 4L,Rampe 1-5,14,2,2,2019-07-07
2019-07-07 22:09:29,02001560,060 4B,Einstreu 1-5,12,2,1,2019-07-07
2019-07-07 22:09:29,020015C6,067 4L,Einstreu 1-5,14,2,1,2019-07-07
2019-07-07 22:09:29,02001560,060 4B,Rampe 1-5,12,2,2,2019-07-07
2019-07-07 22:09:29,020015C6,067 4L,Rampe 1-5,14,2,2,2019-07-07


In [16]:
df[rowSums(is.na(df)) == ncol(df)-1, ]

,Timestamp,Serialnumber_Tag,TagID_Pen_HenID,Zone,Signal,U1,U2,day
NA,NA,NA,NULL,NA,NA,NA,NA,NA


In [17]:
df = df[rowSums(is.na(df)) != ncol(df)-1, ]

In [18]:
#add henID and PenID
df$HenID = lapply(df$TagID_Pen_HenID, function(x){paste('hen',strsplit(x,' ')[[1]][2],sep='_')})
df$PenID = lapply(df$TagID_Pen_HenID, function(x){substring( strsplit(x,' ')[[1]][2], 1, 
                                                                    str_length( strsplit(x,' ')[[1]][2])-1)})

In [19]:
print(dim(df))
head(df)

[1] 4644977      10


Timestamp,Serialnumber_Tag,TagID_Pen_HenID,Zone,Signal,U1,U2,day,HenID,PenID
2019-07-07 22:09:28,02001560,060 4B,Rampe 1-5,12,2,2,2019-07-07,hen_4B,4
2019-07-07 22:09:28,020015C6,067 4L,Rampe 1-5,14,2,2,2019-07-07,hen_4L,4
2019-07-07 22:09:29,02001560,060 4B,Einstreu 1-5,12,2,1,2019-07-07,hen_4B,4
2019-07-07 22:09:29,020015C6,067 4L,Einstreu 1-5,14,2,1,2019-07-07,hen_4L,4
2019-07-07 22:09:29,02001560,060 4B,Rampe 1-5,12,2,2,2019-07-07,hen_4B,4
2019-07-07 22:09:29,020015C6,067 4L,Rampe 1-5,14,2,2,2019-07-07,hen_4L,4


#### keep only usefull variables

In [20]:
df = select(df, Timestamp, HenID, Zone, PenID, day)
print(dim(df))
head(df)

[1] 4644977       5


Timestamp,HenID,Zone,PenID,day
2019-07-07 22:09:28,hen_4B,Rampe 1-5,4,2019-07-07
2019-07-07 22:09:28,hen_4L,Rampe 1-5,4,2019-07-07
2019-07-07 22:09:29,hen_4B,Einstreu 1-5,4,2019-07-07
2019-07-07 22:09:29,hen_4L,Einstreu 1-5,4,2019-07-07
2019-07-07 22:09:29,hen_4B,Rampe 1-5,4,2019-07-07
2019-07-07 22:09:29,hen_4L,Rampe 1-5,4,2019-07-07


#### sort by date (timestamp), in case the log files were not open in the right order

In [21]:
#sort by date (timestamp), in case the log files were not open in the right order
df = df[order(df$Timestamp),]
print(dim(df))
head(df)

[1] 4644977       5


Timestamp,HenID,Zone,PenID,day
2019-07-07 22:09:28,hen_4B,Rampe 1-5,4,2019-07-07
2019-07-07 22:09:28,hen_4L,Rampe 1-5,4,2019-07-07
2019-07-07 22:09:29,hen_4B,Einstreu 1-5,4,2019-07-07
2019-07-07 22:09:29,hen_4L,Einstreu 1-5,4,2019-07-07
2019-07-07 22:09:29,hen_4B,Rampe 1-5,4,2019-07-07
2019-07-07 22:09:29,hen_4L,Rampe 1-5,4,2019-07-07


#### remove zone associated to wrong pen

In [22]:
dico_zone_matching <- hash()
dico_zone_matching$'Rampe 1-5' = c('1','2','3','4','5')
dico_zone_matching$'Rampe 6-10' = c('6','7','8','9','10')
dico_zone_matching$'Einstreu 1-5' = c('1','2','3','4','5')
dico_zone_matching$'Einstreu 6-10' = c('6','7','8','9','10')
dico_zone_matching$'Box1' = c('1')
dico_zone_matching$'Box2' = c('1')
dico_zone_matching$'Box3' = c('2')
dico_zone_matching$'Box4' = c('2')
dico_zone_matching$'Box5' = c('3')
dico_zone_matching$'Box6' = c('3')
dico_zone_matching$'Box7' = c('4')
dico_zone_matching$'Box8' = c('4')
dico_zone_matching$'Box9' = c('5')
dico_zone_matching$'Box10' = c('5')
dico_zone_matching$'Box11' = c('6')
dico_zone_matching$'Box12' = c('6')
dico_zone_matching$'Box13' = c('7')
dico_zone_matching$'Box14' = c('7')
dico_zone_matching$'Box15' = c('8')
dico_zone_matching$'Box16' = c('8')
dico_zone_matching$'Box17' = c('9')
dico_zone_matching$'Box18' = c('9')
dico_zone_matching$'Box19' = c('10')
dico_zone_matching$'Box20' = c('10')
dico_zone_matching$' ' = c('1','2','3','4','5','6','7','8','9','10')

In [23]:
df$Zone[is.na(df$Zone)] == ' '
head(df)

logical(0)

Timestamp,HenID,Zone,PenID,day
2019-07-07 22:09:28,hen_4B,Rampe 1-5,4,2019-07-07
2019-07-07 22:09:28,hen_4L,Rampe 1-5,4,2019-07-07
2019-07-07 22:09:29,hen_4B,Einstreu 1-5,4,2019-07-07
2019-07-07 22:09:29,hen_4L,Einstreu 1-5,4,2019-07-07
2019-07-07 22:09:29,hen_4B,Rampe 1-5,4,2019-07-07
2019-07-07 22:09:29,hen_4L,Rampe 1-5,4,2019-07-07


In [24]:
start = Sys.time()
df$test = apply( df[ , c('PenID', 'Zone') ] , 1 , paste , collapse = "/-/" )
df$is_correct_pen4zone = lapply(df$test, 
                                function(x){strsplit(x,'/-/')[[1]][1] %in% dico_zone_matching[[strsplit(x,'/-/')[[1]][2]]]})
df = select(df, -test)
end = Sys.time()
print(paste0("Total running time: ", (end-start)/60," mn"))

[1] "Total running time: 0.0355405024687449 mn"


In [25]:
df[,c('HenID','PenID','is_correct_pen4zone')] = sapply(df[,c('HenID','PenID','is_correct_pen4zone')],unlist)
df_corr = filter(df, is_correct_pen4zone==FALSE)
write.csv(df_corr, file.path(path_extracted_data, "df_Zone_associated_to_wrong_Pen_all_situation.csv"))
print(dim(df_corr))
head(df_corr)

[1] 2718    6


Timestamp,HenID,Zone,PenID,day,is_correct_pen4zone
2019-07-08 05:23:28,hen_1H,Rampe 6-10,1,2019-07-08,FALSE
2019-07-08 08:01:01,hen_6C,Rampe 1-5,6,2019-07-08,FALSE
2019-07-08 08:35:01,hen_9H,Rampe 1-5,9,2019-07-08,FALSE
2019-07-08 08:56:57,hen_9M,Rampe 1-5,9,2019-07-08,FALSE
2019-07-08 09:35:26,hen_9M,Rampe 1-5,9,2019-07-08,FALSE
2019-07-08 10:11:09,hen_8X,Rampe 1-5,8,2019-07-08,FALSE


In [26]:
print(paste0('We removed ',dim(df_corr)[[1]],' records which zone was associated to a wrong pen'))
df = filter(df, is_correct_pen4zone==TRUE)
df = select(df, -is_correct_pen4zone)
print(dim(df))
head(df)

[1] "We removed 2718 records which zone was associated to a wrong pen"
[1] 4642259       5


Timestamp,HenID,Zone,PenID,day
2019-07-07 22:09:28,hen_4B,Rampe 1-5,4,2019-07-07
2019-07-07 22:09:28,hen_4L,Rampe 1-5,4,2019-07-07
2019-07-07 22:09:29,hen_4B,Einstreu 1-5,4,2019-07-07
2019-07-07 22:09:29,hen_4L,Einstreu 1-5,4,2019-07-07
2019-07-07 22:09:29,hen_4B,Rampe 1-5,4,2019-07-07
2019-07-07 22:09:29,hen_4L,Rampe 1-5,4,2019-07-07


In [27]:
#df[,c('HenID','PenID')] = sapply(df[,c('HenID','PenID')],unlist)
print(dim(df))
head(df)

[1] 4642259       5


Timestamp,HenID,Zone,PenID,day
2019-07-07 22:09:28,hen_4B,Rampe 1-5,4,2019-07-07
2019-07-07 22:09:28,hen_4L,Rampe 1-5,4,2019-07-07
2019-07-07 22:09:29,hen_4B,Einstreu 1-5,4,2019-07-07
2019-07-07 22:09:29,hen_4L,Einstreu 1-5,4,2019-07-07
2019-07-07 22:09:29,hen_4B,Rampe 1-5,4,2019-07-07
2019-07-07 22:09:29,hen_4L,Rampe 1-5,4,2019-07-07


#### handle flickering situations

In [28]:
#add next duration variable
li_df = list()
for (h in unique(df$HenID)){
    df_hen = filter(df, HenID==h)
    #as the next record date (sort by date, then simply shift by one row and add nan at then end)
    df_hen = df_hen[order(df_hen$Timestamp),]
    #same date, one must take the last recorded one & sorting by date might change it. Also it already should be sorted by date
    df_hen$next_record_date = append(as.vector(df_hen['Timestamp'])[[1]][2:dim(df_hen)[1]], 
                                     as.POSIXct('1999-01-01 00:00:00', format="%Y-%m-%d %H:%M:%S"))
    df_hen$duration = df_hen$next_record_date-df_hen$Timestamp
    df_hen$duration = as.integer(df_hen$duration)

    #compute the last record date in order to put interzone also when the duration is >=nbr_sec
    #NA induce numeric instead os POSIXct format for rows starting with NA
    df_hen$previous_record_date = append(as.POSIXct('1999-01-01 00:00:00', format="%Y-%m-%d %H:%M:%S"), 
                                         as.vector(df_hen['Timestamp'])[[1]][1:(dim(df_hen)[1]-1)])

    #compute previous duration in order to put interzone also when the duration is >=nbr_sec
    df_hen$previous_duration = append(c(NA), as.vector(df_hen['duration'])[[1]][1:(dim(df_hen)[1]-1)])
    df_hen$previous_duration = as.integer(df_hen$previous_duration)
    li_df = append(li_df,list(df_hen))
}

#put again in one dataframe
df = bind_rows(li_df)
print(dim(df))
#dont care about the false positive warning
head(df)

[1] 4642259       9


Timestamp,HenID,Zone,PenID,day,next_record_date,duration,previous_record_date,previous_duration
2019-07-07 22:09:28,hen_4B,Rampe 1-5,4,2019-07-07,2019-07-07 22:09:29,1,1999-01-01 00:00:00,NA
2019-07-07 22:09:29,hen_4B,Einstreu 1-5,4,2019-07-07,2019-07-07 22:09:29,0,2019-07-07 22:09:28,1
2019-07-07 22:09:29,hen_4B,Rampe 1-5,4,2019-07-07,2019-07-07 22:09:30,1,2019-07-07 22:09:29,0
2019-07-07 22:09:30,hen_4B,Einstreu 1-5,4,2019-07-07,2019-07-07 22:09:31,1,2019-07-07 22:09:29,1
2019-07-07 22:09:31,hen_4B,Rampe 1-5,4,2019-07-07,2019-07-07 22:09:32,1,2019-07-07 22:09:30,1
2019-07-07 22:09:32,hen_4B,Einstreu 1-5,4,2019-07-07,2019-07-07 22:09:32,0,2019-07-07 22:09:31,1


In [29]:
tail(df)

,Timestamp,HenID,Zone,PenID,day,next_record_date,duration,previous_record_date,previous_duration
4642254,2019-08-22 11:30:35,hen_3L,Rampe 1-5,3,2019-08-22,2019-08-22 11:30:36,1,2019-08-22 11:30:35,0
4642255,2019-08-22 11:30:36,hen_3L,Einstreu 1-5,3,2019-08-22,2019-08-22 11:30:37,1,2019-08-22 11:30:35,1
4642256,2019-08-22 11:30:37,hen_3L,Rampe 1-5,3,2019-08-22,2019-08-22 11:30:38,1,2019-08-22 11:30:36,1
4642257,2019-08-22 11:30:38,hen_3L,Einstreu 1-5,3,2019-08-22,2019-08-22 11:30:38,0,2019-08-22 11:30:37,1
4642258,2019-08-22 11:30:38,hen_3L,Rampe 1-5,3,2019-08-22,2019-08-22 11:30:39,1,2019-08-22 11:30:38,0
4642259,2019-08-22 11:30:39,hen_3L,Einstreu 1-5,3,2019-08-22,1999-01-01 00:00:00,-651321039,2019-08-22 11:30:38,1


In [30]:
#name it interzone when duration is less than 2 seconds
#note that there is no need to merge interzone in one timestamp as we will in any case extend to a time serie for analysis and
#keep only one second over two
df$Zone_without_flickering = df$Zone
df$Zone_without_flickering = as.character(df$Zone_without_flickering)
df[(!is.na(df$duration)) & (df$duration<nbr_sec), 'Zone_without_flickering'] = 'Interzone'
#if its not interzone (i.e. its duration is longer than 3 seconds) and its previous duration is shorter than 3 seconds,
#then its the end of a flickering situation
df[(!is.na(df$previous_duration)) & (df$previous_duration<nbr_sec) & (df$Zone_without_flickering!='Interzone'),
       'Zone_without_flickering'] = 'Interzone_f'
#df$Zone_without_flickering = factor(df$Zone_without_flickering)
                                                                            
#we wont be doing this, as otherwise if the lasst timestamp are flickering situation, we will miss a zone
    #replace 'Zone_without_flickering' by np.nan if the duration is nan (i.e. if last observation)
#df.loc[pd.isnull(df['duration']),'Zone_without_flickering'] = np.nan

head(df)

Timestamp,HenID,Zone,PenID,day,next_record_date,duration,previous_record_date,previous_duration,Zone_without_flickering
2019-07-07 22:09:28,hen_4B,Rampe 1-5,4,2019-07-07,2019-07-07 22:09:29,1,1999-01-01 00:00:00,NA,Interzone
2019-07-07 22:09:29,hen_4B,Einstreu 1-5,4,2019-07-07,2019-07-07 22:09:29,0,2019-07-07 22:09:28,1,Interzone
2019-07-07 22:09:29,hen_4B,Rampe 1-5,4,2019-07-07,2019-07-07 22:09:30,1,2019-07-07 22:09:29,0,Interzone
2019-07-07 22:09:30,hen_4B,Einstreu 1-5,4,2019-07-07,2019-07-07 22:09:31,1,2019-07-07 22:09:29,1,Interzone
2019-07-07 22:09:31,hen_4B,Rampe 1-5,4,2019-07-07,2019-07-07 22:09:32,1,2019-07-07 22:09:30,1,Interzone
2019-07-07 22:09:32,hen_4B,Einstreu 1-5,4,2019-07-07,2019-07-07 22:09:32,0,2019-07-07 22:09:31,1,Interzone


In [31]:
tail(df)

,Timestamp,HenID,Zone,PenID,day,next_record_date,duration,previous_record_date,previous_duration,Zone_without_flickering
4642254,2019-08-22 11:30:35,hen_3L,Rampe 1-5,3,2019-08-22,2019-08-22 11:30:36,1,2019-08-22 11:30:35,0,Interzone
4642255,2019-08-22 11:30:36,hen_3L,Einstreu 1-5,3,2019-08-22,2019-08-22 11:30:37,1,2019-08-22 11:30:35,1,Interzone
4642256,2019-08-22 11:30:37,hen_3L,Rampe 1-5,3,2019-08-22,2019-08-22 11:30:38,1,2019-08-22 11:30:36,1,Interzone
4642257,2019-08-22 11:30:38,hen_3L,Einstreu 1-5,3,2019-08-22,2019-08-22 11:30:38,0,2019-08-22 11:30:37,1,Interzone
4642258,2019-08-22 11:30:38,hen_3L,Rampe 1-5,3,2019-08-22,2019-08-22 11:30:39,1,2019-08-22 11:30:38,0,Interzone
4642259,2019-08-22 11:30:39,hen_3L,Einstreu 1-5,3,2019-08-22,1999-01-01 00:00:00,-651321039,2019-08-22 11:30:38,1,Interzone


In [32]:
head(df_hen)

Timestamp,HenID,Zone,PenID,day,next_record_date,duration,previous_record_date,previous_duration
2019-07-08 07:05:53,hen_3L,Rampe 1-5,3,2019-07-08,2019-07-08 07:05:56,3,1999-01-01 00:00:00,NA
2019-07-08 07:05:56,hen_3L,Einstreu 1-5,3,2019-07-08,2019-07-08 07:05:57,1,2019-07-08 07:05:53,3
2019-07-08 07:05:57,hen_3L,Rampe 1-5,3,2019-07-08,2019-07-08 07:05:57,0,2019-07-08 07:05:56,1
2019-07-08 07:05:57,hen_3L,Einstreu 1-5,3,2019-07-08,2019-07-08 07:05:58,1,2019-07-08 07:05:57,0
2019-07-08 07:05:58,hen_3L,Rampe 1-5,3,2019-07-08,2019-07-08 07:05:59,1,2019-07-08 07:05:57,1
2019-07-08 07:05:59,hen_3L,Einstreu 1-5,3,2019-07-08,2019-07-08 07:06:02,3,2019-07-08 07:05:58,1


In [33]:
unique(df$Zone_without_flickering)

[1] "Interzone"     "Interzone_f"   "Rampe 1-5"     "Einstreu 1-5" 
 [5] "Box8"          "Box7"          "Box1"          "Box2"         
 [9] "Box3"          "Box4"          "Rampe 6-10"    "Einstreu 6-10"
[13] "Box12"         "Box11"         "Box16"         "Box15"        
[17] "Box13"         "Box14"         "Box20"         "Box19"        
[21] "Box5"          "Box6"          "Box18"         "Box17"        
[25] "Box9"          "Box10"

In [34]:
#differentiate between different interzones
#very long as we need to look at the next zone only once we have changed the next row
start = Sys.time()
li_df = list()
for (h in unique(df$HenID)){
    df_hen = filter(df, HenID==h)
    df_hen$Zone = as.character(df_hen$Zone)
    df_hen$interzone_info = df_hen$Zone
    df_hen$interzone_info = as.character(df_hen$interzone_info)
    df_hen$Zone_without_flickering = as.character(df_hen$Zone_without_flickering)
    df_hen$ToRemove = FALSE
    #as we will keep the first entry of consecutives equal zones, we will take the value from the next record to the previous 
    #one (and not the opposite way)
    #idea: keep the first interzone entry, and put all info about these interzones in the interzone_info column
    #start with the last row, and put the rule: if row is not the same as the one after then just add the actual zone as info,
    #otherwise if its the same then add the interzone info of now and after
    for (i in rev(2:(dim(df_hen)[1]-1))){
        xp = df_hen[i-1,]
        x0 = df_hen[i,]
        x1 = df_hen[i+1,]
        if ((x0$Zone_without_flickering!=x1$Zone_without_flickering) & (x1$Zone_without_flickering!='Interzone_f')){
            df_hen[i,'interzone_info'] = x0$Zone
        }
        else {
            df_hen[i,'interzone_info'] = paste0(x0$interzone_info,', ',x1$interzone_info)
        }
        #if its interzone and the previous one is interzone then remove it
        if (((x0$Zone_without_flickering=='Interzone')|(x0$Zone_without_flickering=='Interzone_f')) & (xp$Zone_without_flickering=='Interzone')){
            df_hen[i,'ToRemove'] = TRUE
        }
    }
    
    #df_hen$interzone_info = factor(df_hen$interzone_info)
    li_df = append(li_df,list(df_hen))
}
    
#put again in one dataframe
df = bind_rows(li_df)
print(dim(df))
#dont care about the false positive warning
head(df)
end = Sys.time()
print(paste0("Total running time: ", (end-start)/60," mn"))

[1] 4642259      12


Timestamp,HenID,Zone,PenID,day,next_record_date,duration,previous_record_date,previous_duration,Zone_without_flickering,interzone_info,ToRemove
2019-07-07 22:09:28,hen_4B,Rampe 1-5,4,2019-07-07,2019-07-07 22:09:29,1,1999-01-01 00:00:00,NA,Interzone,Rampe 1-5,FALSE
2019-07-07 22:09:29,hen_4B,Einstreu 1-5,4,2019-07-07,2019-07-07 22:09:29,0,2019-07-07 22:09:28,1,Interzone,"Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5",TRUE
2019-07-07 22:09:29,hen_4B,Rampe 1-5,4,2019-07-07,2019-07-07 22:09:30,1,2019-07-07 22:09:29,0,Interzone,"Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5",TRUE
2019-07-07 22:09:30,hen_4B,Einstreu 1-5,4,2019-07-07,2019-07-07 22:09:31,1,2019-07-07 22:09:29,1,Interzone,"Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5",TRUE
2019-07-07 22:09:31,hen_4B,Rampe 1-5,4,2019-07-07,2019-07-07 22:09:32,1,2019-07-07 22:09:30,1,Interzone,"Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5",TRUE
2019-07-07 22:09:32,hen_4B,Einstreu 1-5,4,2019-07-07,2019-07-07 22:09:32,0,2019-07-07 22:09:31,1,Interzone,"Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5",TRUE


[1] "Total running time: 0.0233528860056842 mn"


In [35]:
tail(df)

,Timestamp,HenID,Zone,PenID,day,next_record_date,duration,previous_record_date,previous_duration,Zone_without_flickering,interzone_info,ToRemove
4642254,2019-08-22 11:30:35,hen_3L,Rampe 1-5,3,2019-08-22,2019-08-22 11:30:36,1,2019-08-22 11:30:35,0,Interzone,"Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5",TRUE
4642255,2019-08-22 11:30:36,hen_3L,Einstreu 1-5,3,2019-08-22,2019-08-22 11:30:37,1,2019-08-22 11:30:35,1,Interzone,"Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5",TRUE
4642256,2019-08-22 11:30:37,hen_3L,Rampe 1-5,3,2019-08-22,2019-08-22 11:30:38,1,2019-08-22 11:30:36,1,Interzone,"Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5",TRUE
4642257,2019-08-22 11:30:38,hen_3L,Einstreu 1-5,3,2019-08-22,2019-08-22 11:30:38,0,2019-08-22 11:30:37,1,Interzone,"Einstreu 1-5, Rampe 1-5, Einstreu 1-5",TRUE
4642258,2019-08-22 11:30:38,hen_3L,Rampe 1-5,3,2019-08-22,2019-08-22 11:30:39,1,2019-08-22 11:30:38,0,Interzone,"Rampe 1-5, Einstreu 1-5",TRUE
4642259,2019-08-22 11:30:39,hen_3L,Einstreu 1-5,3,2019-08-22,1999-01-01 00:00:00,-651321039,2019-08-22 11:30:38,1,Interzone,Einstreu 1-5,FALSE


In [37]:
#correct names and keep the first interzone row
df$Zone_without_flickering_nodiff = df$Zone_without_flickering
df$interzone_name = lapply(df$interzone_info, function(x){paste0('Interzone_', paste(sort(unlist(lapply(unique(lapply(strsplit(x,','), function(x){trimws(x)})[[1]]),
      function(x){substr(x, start = 1, stop = 1)}))), collapse = ''))})
df$Zone_without_flickering = ifelse(df$Zone_without_flickering=='Interzone', df$interzone_name, df$Zone_without_flickering)

In [39]:
#we remove NOW the interzones record that are not the first record and NOT BEFOREHAND, in order to verify things all together
#remove interzone that are not the first one
x0 = dim(df)[1]
df = filter(df, df$ToRemove==FALSE)
#Now, duration/previous_record_date/next_record_date makes no more sense for interzone and consecutives equal records, so lets 
#remove it
df = select(df, -duration,-previous_duration,-previous_record_date,-ToRemove,-Zone_without_flickering_nodiff,
       -interzone_name)
#SHOULD BE: We remove 3291327 records due to flickering situations, we are left now with 1350932 records
print(paste0('We remove ',x0-dim(df)[1],' records due to flickering situations, we are left now with ',dim(df)[1], ' records'))

[1] "We remove 3291327 records due to flickering situations, we are left now with 1350932 records"


#### remove consecutives equal Zone for same hens at (not strictly) more than nbr_second second duration ?

In [40]:
#add next zone based on Zone_without_flickering (for quality verification)
li_df = list()
for (h in unique(df$HenID)){
    df_hen = filter(df, HenID==h)
    #as the next record date (sort by date, then simply shift by one row and add nan at then end)
    df_hen = df_hen[order(df_hen$Timestamp),]
    #same date, one must take the last recorded one & sorting by date might change it. Also it already shoul dbe sorted by date
    df_hen$next_zone = append(as.vector(df_hen['Zone_without_flickering'])[[1]][2:dim(df_hen)[1]], '')
    li_df = append(li_df,list(df_hen))
}
   
#put again in one dataframe
df = bind_rows(li_df)
print(dim(df))
#dont care about the false positive warning
tail(df)

[1] 1350932       9


,Timestamp,HenID,Zone,PenID,day,next_record_date,Zone_without_flickering,interzone_info,next_zone
1350927,2019-08-22 11:29:52,hen_3L,Rampe 1-5,3,2019-08-22,2019-08-22 11:30:02,Rampe 1-5,Rampe 1-5,Interzone_ER
1350928,2019-08-22 11:30:02,hen_3L,Einstreu 1-5,3,2019-08-22,2019-08-22 11:30:02,Interzone_ER,"Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5",Interzone_ER
1350929,2019-08-22 11:30:21,hen_3L,Einstreu 1-5,3,2019-08-22,2019-08-22 11:30:21,Interzone_ER,"Einstreu 1-5, Rampe 1-5",Einstreu 1-5
1350930,2019-08-22 11:30:27,hen_3L,Einstreu 1-5,3,2019-08-22,2019-08-22 11:30:32,Einstreu 1-5,Einstreu 1-5,Interzone_ER
1350931,2019-08-22 11:30:32,hen_3L,Rampe 1-5,3,2019-08-22,2019-08-22 11:30:32,Interzone_ER,"Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5, Rampe 1-5, Einstreu 1-5",Interzone_E
1350932,2019-08-22 11:30:39,hen_3L,Einstreu 1-5,3,2019-08-22,1999-01-01 00:00:00,Interzone_E,Einstreu 1-5,


In [43]:
head(df)

Timestamp,HenID,Zone,PenID,day,next_record_date,Zone_without_flickering,interzone_info,next_zone,correction_is_consecutive_equal_zone,is_flick
2019-07-07 22:09:28,hen_4B,Rampe 1-5,4,2019-07-07,2019-07-07 22:09:29,Interzone_R,Rampe 1-5,Interzone_ER,FALSE,TRUE
2019-07-07 22:10:15,hen_4B,Rampe 1-5,4,2019-07-07,2019-07-07 22:10:16,Interzone_ER,"Rampe 1-5, Einstreu 1-5",Interzone_ER,FALSE,TRUE
2019-07-07 22:11:06,hen_4B,Rampe 1-5,4,2019-07-07,2019-07-07 22:11:07,Interzone_ER,"Rampe 1-5, Einstreu 1-5",Interzone_ER,FALSE,TRUE
2019-07-07 22:11:57,hen_4B,Rampe 1-5,4,2019-07-07,2019-07-07 22:11:58,Interzone_ER,"Rampe 1-5, Einstreu 1-5",Rampe 1-5,FALSE,TRUE
2019-07-07 22:17:23,hen_4B,Rampe 1-5,4,2019-07-07,2019-07-07 22:17:29,Rampe 1-5,Rampe 1-5,Einstreu 1-5,FALSE,FALSE
2019-07-07 22:17:29,hen_4B,Einstreu 1-5,4,2019-07-07,2019-07-07 22:18:49,Einstreu 1-5,Einstreu 1-5,Interzone_ER,FALSE,FALSE


In [ ]:
#True if next zone is equal to the actual zone (Zone_without_flickering) 
df$correction_is_consecutive_equal_zone = FALSE
#df$Zone_without_flickering[is.na(df$Zone_without_flickering)] = ''
df$Zone_without_flickering = as.character(df$Zone_without_flickering)
df$is_flick = ifelse(df$Zone_without_flickering!='',startsWith(df$Zone_without_flickering,'Interzone'), FALSE)
df[(df$next_zone==df$Zone_without_flickering) & (df$is_flick==FALSE), 'correction_is_consecutive_equal_zone'] = TRUE
df$next_zone = as.character(df$next_zone)

In [52]:
df_test = filter(df, df$correction_is_consecutive_equal_zone==TRUE)
print(paste0('There is ',dim(df_test)[1],' records that has same zone than the previous one without being a flickering situation, \
we wont remove these records from the clean_record file as it wont change anything for the time serie'))
#should be: There is 1863 records that has same zone than the previous one without being a flickering situation, ....
#TODO: why not the same as in python? check nan et 1999 date
df_test

[1] "There is 1852 records that has same zone than the previous one without being a flickering situation, \nwe wont remove these records from the clean_record file as it wont change anything for the time serie"


Timestamp,HenID,Zone,PenID,day,next_record_date,Zone_without_flickering,interzone_info,next_zone,correction_is_consecutive_equal_zone,is_flick
2019-07-13 14:45:12,hen_4B,Rampe 1-5,4,2019-07-13,2019-07-13 14:45:15,Rampe 1-5,"Rampe 1-5, Rampe 1-5",Rampe 1-5,TRUE,FALSE
2019-07-14 06:22:10,hen_4B,Einstreu 1-5,4,2019-07-14,2019-07-14 06:24:40,Einstreu 1-5,"Einstreu 1-5, Einstreu 1-5",Einstreu 1-5,TRUE,FALSE
2019-07-28 08:11:26,hen_4B,Einstreu 1-5,4,2019-07-28,2019-07-28 08:11:59,Einstreu 1-5,"Einstreu 1-5, Einstreu 1-5",Einstreu 1-5,TRUE,FALSE
2019-07-28 13:08:55,hen_4B,Einstreu 1-5,4,2019-07-28,2019-07-28 13:19:15,Einstreu 1-5,"Einstreu 1-5, Einstreu 1-5",Einstreu 1-5,TRUE,FALSE
2019-07-29 01:53:22,hen_4B,Einstreu 1-5,4,2019-07-29,2019-07-29 01:53:26,Einstreu 1-5,"Einstreu 1-5, Einstreu 1-5",Einstreu 1-5,TRUE,FALSE
2019-08-02 17:30:18,hen_4B,Einstreu 1-5,4,2019-08-02,2019-08-02 17:30:48,Einstreu 1-5,"Einstreu 1-5, Einstreu 1-5",Einstreu 1-5,TRUE,FALSE
2019-08-10 12:12:47,hen_4B,Einstreu 1-5,4,2019-08-10,2019-08-10 12:14:07,Einstreu 1-5,"Einstreu 1-5, Einstreu 1-5",Einstreu 1-5,TRUE,FALSE
2019-08-15 11:55:11,hen_4B,Einstreu 1-5,4,2019-08-15,2019-08-15 11:56:50,Einstreu 1-5,"Einstreu 1-5, Einstreu 1-5",Einstreu 1-5,TRUE,FALSE
2019-08-16 12:07:00,hen_4B,Rampe 1-5,4,2019-08-16,2019-08-16 12:07:04,Rampe 1-5,"Rampe 1-5, Rampe 1-5",Rampe 1-5,TRUE,FALSE
2019-07-08 02:20:06,hen_4L,Rampe 1-5,4,2019-07-08,2019-07-08 02:20:10,Rampe 1-5,"Rampe 1-5, Rampe 1-5",Rampe 1-5,TRUE,FALSE


In [53]:
#save
write.csv(select(df_test, HenID, Timestamp, Zone_without_flickering, next_record_date, next_zone),
          file.path(path_extracted_data, "df_consecutives_equal_zone.csv"))

### Save

In [55]:
write.csv(select(df, -next_record_date, -next_zone), file.path(path_extracted_data, "df_clean_records.csv"))

In [56]:
END_ALL = Sys.time()
print(paste0("TOTAL RUNNING TIME: ", (END_ALL-START_ALL)/60," mn"))

[1] "TOTAL RUNNING TIME: 0.035206996642881 mn"


# Time series